# DSC 680
## Assignment 8.1
## Chase Lemons

In [59]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

import datetime as dt
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
import timeit

In [2]:
df = pd.read_csv("indeed_job_dataset.csv", sep =",")

In [3]:
df.head()

,Unnamed: 0,Job_Title,Link,Queried_Salary,Job_Type,Skill,No_of_Skills,Company,No_of_Reviews,No_of_Stars,...,MD,DC,NC,Other_states,Consulting and Business Services,Internet and Software,Banks and Financial Services,Health Care,Insurance,Other_industries
0,0,Data Scientist,https://www.indeed.com/rc/clk?jk=6a105f495c36a...,<80000,data_scientist,"['SAP', 'SQL']",2,Express Scripts,3301.0,3.3,...,0,0,0,1,0,0,0,1,0,0
1,1,Data Scientist,https://www.indeed.com/rc/clk?jk=86afd561ea8c6...,<80000,data_scientist,"['Machine Learning', 'R', 'SAS', 'SQL', 'Python']",5,Money Mart Financial Services,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2,Data Scientist,https://www.indeed.com/rc/clk?jk=e0aad317e6d45...,<80000,data_scientist,"['Data Mining', 'Data Management', 'R', 'SAS',...",9,comScore,62.0,3.5,...,0,0,0,1,0,0,0,0,0,0
3,3,Graduate Studies Program - Data Scientist,https://www.indeed.com/rc/clk?jk=1cfdd9e391a63...,<80000,data_scientist,['Certified Internal Auditor'],1,Central Intelligence Agency,158.0,4.3,...,0,1,0,0,0,0,0,0,0,1
4,4,Data Scientist I,https://www.indeed.com/rc/clk?jk=fec647775a21e...,<80000,data_scientist,"['Statistical Software', 'Time Management', 'R...",7,Federal Reserve Bank of Dallas,495.0,4.1,...,0,0,0,0,0,0,1,0,0,0


In [12]:
import docx2txt
my_text = docx2txt.process("Chase Lemons Resume 6_17_2019.docx")


Chase H. Lemons

	Advanced Analyst  

	509.668.8358  

	Clemons_12@hotmail.com

	Goal oriented professional with 5+ years of experience in big data and advanced analytics across multiple functions. Seeking a position to utilize and enhance my leadership, analytics and developer skills. Extremely passionate about analytics, strategy and process improvement.

	







Work Experience:





Sr. Advanced Analyst | Textron Aviation	            				         October 2018 - Present

• Actively provide strategy for overall analytics roadmap and operating model for Textron Aviation.

• Lead a Tableau proof of concept with the business.

• Develop dashboards for multiple business areas utilizing Tableau, Qlikview, and Power BI.

• Ensure ETL processes in and out of the enterprise data warehouse perform successfully. 

• Uses a combination of R, Python and SQL for data manipulation, analysis and reporting daily.

• Implemented a model to identify parts that needed price adjustment based on ecommer

In [56]:
import re
content = []
for line in my_text.splitlines():
  if line != '':
    clean_lines = line.strip().lower()
    clean_lines2 = re.sub(r'[^a-zA-Z0-9\._-]', ' ', clean_lines)
    clean_lines2 = re.sub(r'\s+', ' ', clean_lines2)
    clean_lines2 = clean_lines2.strip()
    content.append(clean_lines2)
    
jobs = df["Description"].tolist()

for i in range(len(jobs)):
    jobs[i] = str(jobs[i]).strip().lower()

def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element)
        result += " "
    return result

resume = concatenate_list_data(content)

In [61]:
def cos_sim(a,b):
    calc = dot(a, b)/(norm(a)*norm(b))
    return calc


cos_sim_score = []
included_list = ['NN','NNP', 'NNPS', 'NNS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

resume_tolk = word_tokenize(resume)
l1 =[]

tokenized_can_1 = []
aa = nltk.pos_tag(resume_tolk)
filt_sent = []
for k in aa:
    if k[1] in included_list:
        filt_sent.append(k[0].lower())
    tokenized_can_1.append(filt_sent)
    X_set = set(tokenized_can_1[0])


req_1 = df["Description"].values



In [96]:
%%time
cos_sim_score = []
for i in range(len(jobs)):

    job_des = jobs[i]
  
    # tokenization   
    job_des_tolk = word_tokenize(job_des) 

    l1 =[];l2 =[]  

    tokenized_req_1 = []
    bb = nltk.pos_tag(job_des_tolk)
    filt_sent = []
    for k in bb:
        if k[1] in included_list:
            filt_sent.append(k[0].lower())
    tokenized_req_1.append(filt_sent)
    Y_set = set(tokenized_req_1[0])

    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector 
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 

    cos = cos_sim(l1,l2)
    cos_sim_score.append(cos)
    df["des_res_similarity"] = cos_sim_score

Wall time: 8min 35s


In [101]:
df.to_csv("indeed_job_dataset_with_similarity.csv")